In [6]:
%load_ext autoreload
%autoreload 2

from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import uuid
from modules.db_operations import DatabaseConnector
from modules.sends import send
EMAIL_ADDRESS_FROM = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet


pd.set_option('display.max_columns', None)
#This can be further cleansed in the future
df = pd.read_csv('KC SCHOOLS - KC SCHOOLS.csv')


#Temporary testing to modify the Athletic Director email to my own
df.loc[0, 'Athletic Director.1'] = '2015samtaylor@gmail.com'
df.loc[1, 'Athletic Director.1'] = 'sammytaylor2006@gmail.com'
df.loc[2, 'Athletic Director.1'] = 'jonjones3456@gmail.com'
df.loc[3, 'Athletic Director.1'] = 'sammytaylor2006@gmail.com'
df.loc[3, 'Athletic Director.1'] = 'jerrybons2006@gmail.com'


def process(contact_column, sport, email_campaign_name):

    data_list = []
    processed_emails = set()

    for index, row in df[:5].iterrows():

        # Get the current date
        now = datetime.now()
        formatted_date = now.strftime("%m%d%Y%H%M%S")

        school = row['School Name']
        png_path = row['PNG_PATH']
        contact = row[contact_column]
         #estiablish unique identifier, in order to match up FK relation, and merges
        unique_identifier = str(uuid.uuid4())

        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date, unique_identifier]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)
        
        send.send_mail(EMAIL_ADDRESS_FROM, EMAIL_PASS, school, sport, png_path, contact, unique_identifier)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent', 'recipient_id'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date_sent'] = data_list['date_sent'].astype(str)

    data_list['contact_position'] = contact_column
    data_list['sent_from'] = EMAIL_ADDRESS_FROM
    data_list['email_campaign'] = email_campaign_name
    data_list['date_sent'] = now.strftime("%Y-%m-%d %H:%M:%S")

    data_list.to_csv('output.csv', index=False)

    return(data_list)

# #Must be the column name of the position wanting to send, sport fills in email headers, third arg is campaign name
email_history = process('Athletic Director.1', 'Basketball', 'Initial')

# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

# Instantiate the DatabaseConnector class, and send over the dataframe via append
db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
db_connector.send(email_history)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\modules\db_operations.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


In [4]:
pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\KC SCHOOLS - KC SCHOOLS.csv')

,School District,Level,School Name,LOGO,OUR LOGO,School Search,PHONE SEARCH,Unnamed: 7,Principal,Athletic Director,Vice-Principal,Football Coach,Baseball Coach,Softball Coach,Basketball Coach,Soccer Coach,Address Search,Address,Principal.1,Athletic Director.1,Vice-Principal.1,Football Coach.1,Baseball Coach.1,Softball Coach.1,Basketball Coach.1,Soccer Coach.1,Unnamed: 26,email replacement,emails wasn't delivered,yellow did not remove,PNG_PATH
0,KCK US-500,High School,F.L. Schlagle High School (KCKS),https://www.google.com/search?client=firefox-b...,https://customplanet.com/tshirtecommerce/uploa...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,NaN,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,"2214 N 59 St, Kansas City, KS 66104,USA",Yolanda.Thompson@kckps.org,Sonya.Scott@kckps.org,Amaya@kckps.org,taylor.wallacejr@kckps.org,NaN,NaN,"dagonza@kckps.org, Mark.western@kckps.org",skylar.roush@kckps.org,NaN,ABLawless@bluevalleyk12.org(JHayes01@bluevalle...,heata@insightschools.net,NaN,0 F.L. Schlagle High School (KCKS)
1,KCK US-500,Middle School,Plaza Middle (Park Hill),https://www.google.com/search?client=firefox-b...,https://customplanet.com/tshirtecommerce/uploa...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,NaN,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,"6501 Nw 72nd St, Kansas City MO 64151",wrightt@parkhill.k12.mo.us,davisj@parkhill.k12.mo.us,danielj@parkhill.k12.mo.us,NaN,NaN,NaN,NaN,NaN,NaN,Tim.Newton@usd469.net(mindy.wells@usd469.net),greg.johnson@lsr7.net,NaN,1 Plaza Middle (Park Hill)
2,Kearney School District,Middle School,Kearney Middle School,https://www.google.com/search?client=firefox-b...,https://customplanet.com/tshirtecommerce/uploa...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,NaN,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,"200 East 5th St, Kearney MO 64060",woodsb@ksdr1.net,campbellk@ksdr1.net,nolande@ksdr1.net,NaN,NaN,NaN,NaN,NaN,NaN,LongeneckerD@parkhill.k12.mo.us(herrenk@parkhi...,johnny.dodge@grandviewc4.net,NaN,2 Kearney Middle School
3,Smithville School District,Middle School,Smithville Middle School,https://www.google.com/search?client=firefox-b...,https://customplanet.com/tshirtecommerce/uploa...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,NaN,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https://www.google.com/search?client=firefox-b...,https

# To DO

In [ ]:
#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 


#Think about how the scrape is going to work in thread. 
#There is data within there right now. What if the foreign key, does not match?
#Need some SSIS scripts to drop duplicates, etc... more focus on SSIS once data is in

In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\KC SCHOOLS - KC SCHOOLS.csv')

In [ ]:
df